In [2]:
!git clone "https://github.com/angel-one/smartapi-python.git"

fatal: destination path 'smartapi-python' already exists and is not an empty directory.


In [3]:
!pip install -r ./smartapi-python/requirements_dev.txt

  Using cached autobahn-23.6.2-cp310-cp310-win_amd64.whl
  Using cached Automat-22.10.0-py2.py3-none-any.whl.metadata (1.0 kB)
  Using cached certifi-2023.5.7-py3-none-any.whl.metadata (2.2 kB)
  Using cached charset_normalizer-3.1.0-cp310-cp310-win_amd64.whl.metadata (31 kB)
  Using cached constantly-15.1.0-py2.py3-none-any.whl.metadata (1.2 kB)
  Using cached cryptography-41.0.1-cp37-abi3-win_amd64.whl.metadata (5.3 kB)
  Using cached hyperlink-21.0.0-py2.py3-none-any.whl.metadata (1.5 kB)
  Using cached incremental-22.10.0-py2.py3-none-any.whl.metadata (6.0 kB)
  Using cached isodate-0.6.1-py2.py3-none-any.whl.metadata (9.6 kB)
  Using cached pycrypto-2.6.1.tar.gz (446 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached pyotp-2.8.0-py3-none-any.whl.metadata (8.8 kB)
  Using cached pyparsing-3.1.0-py3-none-any.whl.metadata (4.9 kB)
  Using cached pytz-2023.3-py2.py3-none-any.whl.metadata (22 kB)
  Using cached rdfli

  error: subprocess-exited-with-error
  
  python setup.py bdist_wheel did not run successfully.
  exit code: 1
  
  [2 lines of output]
  error: Microsoft Visual C++ 14.0 or greater is required. Get it with "Microsoft C++ Build Tools": https://visualstudio.microsoft.com/visual-cpp-build-tools/
  [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for pycrypto
ERROR: Could not build wheels for pycrypto, which is required to install pyproject.toml-based projects


In [13]:
from SmartApi import SmartConnect #or from SmartApi.smartConnect import SmartConnect
import pyotp
from logzero import logger

In [14]:
api_key = ''
username = ''
pwd = ''
smartApi = SmartConnect(api_key)

In [15]:
totp_value = open('TOTP_key.txt',"r").read()

In [16]:
totp_value

'2ILIRHL7GL2KNRWNUMQYU2QAAM'

In [17]:
try:
    # token = "2ILIRHL7GL2KNRWNUMQYU2QAAM"
    token = str(totp_value)
    totp = pyotp.TOTP(token).now()
except Exception as e:
    logger.error("Invalid Token: The provided token is not valid.")
    raise e

In [18]:
correlation_id = "abcde"
data = smartApi.generateSession(username, pwd, totp)

In [ ]:
data

In [20]:
if data['status'] == False:
    logger.error(data)
else:
    # login api call
    # logger.info(f"You Credentials: {data}")
    authToken = data['data']['jwtToken']
    refreshToken = data['data']['refreshToken']
    # fetch the feedtoken
    feedToken = smartApi.getfeedToken()
    # fetch User Profile
    res = smartApi.getProfile(refreshToken)
    smartApi.generateToken(refreshToken)
    res1=res['data']['exchanges']

In [21]:
res1 

['nse_fo', 'nse_cm', 'cde_fo', 'ncx_fo', 'bse_fo', 'bse_cm', 'mcx_fo']

In [22]:
if data['status'] == False:
    logger.error(data)
    
else:
    # login api call
    # logger.info(f"You Credentials: {data}")
    authToken = data['data']['jwtToken']
    refreshToken = data['data']['refreshToken']
    # fetch the feedtoken
    feedToken = smartApi.getfeedToken()
    # fetch User Profile
    res = smartApi.getProfile(refreshToken)
    smartApi.generateToken(refreshToken)
    res=res['data']['exchanges']

    #place order
    try:
        orderparams = {
            "variety": "NORMAL",
            "tradingsymbol": "SBIN-EQ",
            "symboltoken": "3045",
            "transactiontype": "SELL",
            "exchange": "NSE",
            "ordertype": "LIMIT",
            "producttype": "INTRADAY",
            "duration": "DAY",
            "price": "19500",
            "squareoff": "0",
            "stoploss": "0",
            "quantity": "1"
            }
        # Method 1: Place an order and return the order ID
        orderid = smartApi.placeOrder(orderparams)
        logger.info(f"PlaceOrder : {orderid}")
        # Method 2: Place an order and return the full response
        response = smartApi.placeOrderFullResponse(orderparams)
        logger.info(f"PlaceOrder : {response}")
    except Exception as e:
        logger.exception(f"Order placement failed: {e}")

[I 240515 11:12:26 566526955:34] PlaceOrder : 240515000678356
[I 240515 11:12:27 566526955:37] PlaceOrder : {'status': True, 'message': 'SUCCESS', 'errorcode': '', 'data': {'script': 'SBIN-EQ', 'orderid': '240515000678388', 'uniqueorderid': '915367dc-2b71-4f9e-a533-1f3ff44f8a0d'}}


In [11]:
 try:
        historicParam={
        "exchange": "NSE",
        "symboltoken": "3045",
        "interval": "ONE_MINUTE",
        "fromdate": "2021-02-08 09:00", 
        "todate": "2021-02-08 09:16"
        }
        x = smartApi.getCandleData(historicParam)
except Exception as e:
        logger.exception(f"Historic Api failed: {e}")

In [12]:
x

{'status': True,
 'message': 'SUCCESS',
 'errorcode': '',
 'data': [['2021-02-08T09:15:00+05:30', 398.0, 403.45, 397.3, 401.85, 2609877],
  ['2021-02-08T09:16:00+05:30', 402.15, 403.0, 401.45, 401.75, 1470752]]}

In [ ]:
data